In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, classification_report
from keras.models import Sequential, Model 
from keras.layers import Dense, Input, concatenate, Activation, Dropout
from keras.optimizers import Adam
from tensorflow import set_random_seed
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from micromlgen import port

Using TensorFlow backend.


In [2]:
RANDOM_SEED = 42

In [3]:
np.random.seed(RANDOM_SEED)
set_random_seed(RANDOM_SEED)

In [4]:
with open('data/X.pkl', 'rb') as f:
    X = pickle.load(f)

with open('data/y.pkl', 'rb') as f:
    y = pickle.load(f)

Split in train and test sets

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)

In [9]:
#y_train = pd.get_dummies(y_train).values
#y_test = pd.get_dummies(y_test).values

In [10]:
y_train

array([6, 4, 6, 1, 2, 5, 3, 6, 5, 1, 2, 3, 0, 1, 5, 5, 1, 2, 4, 2, 6, 2,
       0, 1, 1, 6, 0, 3, 4, 5, 5, 2, 3, 6, 5, 2, 3, 2, 6, 4, 2, 5, 4, 1,
       3, 1, 5, 0, 1, 0, 1, 1, 3, 2, 0, 0, 4, 0, 0, 4, 0, 4, 4, 6, 3, 3,
       6, 3, 4, 1, 0, 2, 5, 6, 3, 4, 1, 4, 4, 2, 5, 3, 5, 0, 5, 4, 2, 0,
       6, 0, 1, 3, 0, 5, 0, 6, 4, 1, 1, 4, 0, 2, 6, 6, 5, 2, 6, 3, 2, 3,
       0, 3, 0, 3, 5, 6, 0, 2, 4, 2, 0, 1, 6, 2, 0, 1, 6, 3, 1, 2, 4, 1,
       3, 5, 0, 3, 0, 6, 3, 6, 4, 5, 6, 6, 5, 0, 1, 3, 2, 1, 5, 3, 4, 4,
       4, 3, 5, 5, 3, 2, 4, 2, 2, 4, 4, 0, 1, 6, 0, 6, 4, 4, 2, 1, 3, 5,
       3, 3, 3, 4, 2, 4, 5, 1, 2, 2, 5, 6, 6, 3, 1, 1, 2, 3, 2, 0, 1, 3,
       5, 6, 4, 2, 3, 5, 2, 4, 2, 3, 3, 2, 5, 0, 1, 4, 4, 4, 6, 2, 6, 1,
       5, 6, 5, 4, 3, 3, 4, 3, 5, 2, 3, 5, 4, 1, 4, 1, 6, 6, 1, 6, 1, 2,
       0, 4, 1, 4, 6, 3, 6, 4, 6, 2, 1, 1, 4, 4, 4, 5, 2, 2, 3, 0, 4, 0,
       0, 4, 5, 5, 4, 3, 6, 3, 5, 1, 1, 2, 4, 2, 5, 3, 0, 2, 2, 3, 1, 0,
       1, 1, 5, 5, 1, 4, 0, 2, 4, 3, 3, 5, 2, 4, 2,

In [11]:
X_train.shape[1]

5

## Models Evaluation

### RF

In [12]:
model = RandomForestClassifier(n_estimators=5, random_state=42)
#model = MLPClassifier(random_state=42)

In [13]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [14]:
y_pred = model.predict(X_test)

In [15]:
f1_score(y_test, y_pred, average="weighted")

1.0

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        20

    accuracy                           1.00       140
   macro avg       1.00      1.00      1.00       140
weighted avg       1.00      1.00      1.00       140



In [17]:
print(confusion_matrix(y_test, y_pred))

[[20  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 20  0  0  0  0]
 [ 0  0  0 20  0  0  0]
 [ 0  0  0  0 20  0  0]
 [ 0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0 20]]


In [18]:
with open('rf_model.h', 'w') as f:  # change path if needed
    f.write(port(model, optimize=False))

In [19]:
with open('models/rf_model.pkl', 'wb') as f:
    pickle.dump(model, f)


### MLP

In [41]:
# design network

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=32))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


W0526 12:17:26.929143  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0526 12:17:26.941074  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0526 12:17:26.967005  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [42]:
opt = Adam(lr=1e-5, decay=1e-5)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


W0526 12:17:27.637071  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0526 12:17:27.678496  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0526 12:17:27.690463  6712 deprecation.py:323] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [43]:
# fit network
history = model.fit(x=X_train, y=y_train, epochs=100, batch_size=16, validation_split=0.25, verbose=1, shuffle=True)

W0526 12:17:28.686479  6712 deprecation_wrapper.py:119] From C:\Users\giova\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 360 samples, validate on 120 samples
Epoch 1/100
360/360 [==============================] - 0s 961us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 2/100
360/360 [==============================] - 0s 93us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 3/100
360/360 [==============================] - 0s 109us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 4/100
360/360 [==============================] - 0s 89us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 5/100
360/360 [==============================] - 0s 101us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 6/100
360/360 [==============================] - 0s 92us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - val_acc: 0.1583
Epoch 7/100
360/360 [==============================] - 0s 109us/step - loss: -23.4245 - acc: 0.1694 - val_loss: -24.4450 - 

In [89]:
model.save('models/rnn-cnn')

In [ ]:
with open('models/rnn-cnn-history.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')
pyplot.legend()
pyplot.show()

In [187]:
# make a prediction
y_pred = model.predict(X_test)

In [188]:
np.argmax(y_pred, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [101]:
f1_score(y_test, np.argmax(y_pred, axis=1), average="weighted")

AxisError: axis 1 is out of bounds for array of dimension 1

In [81]:
print(classification_report(y_test, np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        20
           3       0.17      1.00      0.29        20
           4       0.00      0.00      0.00        20
           5       0.00      0.00      0.00        20

    accuracy                           0.17       119
   macro avg       0.03      0.17      0.05       119
weighted avg       0.03      0.17      0.05       119



c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [84]:
print(confusion_matrix(y_test, y_pred.argmax(axis=1)))

[[ 0  0  0 19  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]]
